In [6]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np

In [7]:
lambda_vector = [0.0, 0.0, 0.05]
# molecule string for H2O
molecule_string = """

0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
symmetry c1
"""

# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}


In [8]:
calc = CQEDRHFCalculator(lambda_vector, molecule_string, psi4_options)
calc.run()
calc.summary()


Scratch directory: /tmp/

*** tstart() called on Jessicas-iMac.local
*** at Sat Jun 14 15:48:35 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass 

KeyError: -5.297834476375163

In [9]:
calc.summary()

RHF Energy:           -76.02141845 Ha
CQED-RHF Energy:      -76.01635528 Ha
Dipole Energy:        0.00004736 Ha
Nuclear Repulsion:    8.80146556 Ha
Dipole Moment:        [6.44791790e-16 2.10942375e-15 8.24421358e-01]


In [10]:
expected_h2o_e = -76.016355284146
assert np.isclose(expected_h2o_e, calc.cqed_rhf_energy)